In [1]:
#Load in Prerequisite
import sys
! {sys.executable} -m pip install --upgrade 'xdmod-data>=1.0.0,<2.0.0' python-dotenv tabulate
#Configure formatting
import sys
def exception_handler(exception_type, exception, traceback):
    print("%s: %s" % (exception_type.__name__, exception), file=sys.stderr)
get_ipython()._showtraceback = exception_handler
import sys
def exception_handler(exception_type, exception, traceback):
    print("%s: %s" % (exception_type.__name__, exception), file=sys.stderr)
get_ipython()._showtraceback = exception_handler
from IPython.display import display, Markdown
def display_df_md_table(df):
    return display(Markdown(df.replace('\n', '<br/>', regex=True).to_markdown()))
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import xdmod_data.themes
pio.templates.default = "timeseries"

In [1]:
#Set up enviroment and connection
from pathlib import Path
from os.path import expanduser
xdmod_data_env_path = Path(expanduser('~/xdmod-data.env'))
try:
    with open(xdmod_data_env_path):
        pass
except FileNotFoundError:
    with open(xdmod_data_env_path, 'w') as xdmod_data_env_file:
        xdmod_data_env_file.write('XDMOD_API_TOKEN=')
    xdmod_data_env_path.chmod(0o600)
#Check if API key file is present
from dotenv import load_dotenv
load_dotenv(xdmod_data_env_path, override=True)
#Connect to sever 
#!NOTE: Update from test sever later on to generate with latest data and such
from xdmod_data.warehouse import DataWarehouse
dw = DataWarehouse('https://xdmod-dev.ccr.xdmod.org/')

In [2]:
#Create dictionary of all possible callable dates
from datetime import date,timedelta

#yoinked from xdmod-data/xdmod_data/_validator.py
def __date_add_years(old_date, year_delta):
    # Make dates behave like Ext.JS, i.e., if a date is specified
    # with a day value that is too big, add days to the last valid
    # day in that month, e.g., 2023-02-31 becomes 2023-03-03.
    new_date_year = old_date.year + year_delta
    new_date_day = old_date.day
    days_above = 0
    keep_going = True
    while keep_going:
        try:
            new_date = date(new_date_year, old_date.month, new_date_day)
            keep_going = False
        except ValueError:
            new_date_day -= 1
            days_above += 1
    return new_date + timedelta(days=days_above)
def get_dates_from_duration():
    today = date.today()
    yesterday = today + timedelta(days=-1)
    last_week = today + timedelta(days=-7)
    last_month = today + timedelta(days=-30)
    last_quarter = today + timedelta(days=-90)
    this_month_start = date(today.year, today.month, 1)
    if today.month == 1:
        last_full_month_start_year = today.year - 1
        last_full_month_start_month = 12
    else:
        last_full_month_start_year = today.year
        last_full_month_start_month = today.month - 1
    last_full_month_start = date(
        last_full_month_start_year,
        last_full_month_start_month,
        1,
    )
    last_full_month_end = this_month_start + timedelta(days=-1)
    this_quarter_start = date(
        today.year,
        ((today.month - 1) // 3) * 3 + 1,
        1,
    )
    if today.month < 4:
        last_quarter_start_year = today.year - 1
    else:
        last_quarter_start_year = today.year
    last_quarter_start = date(
        last_quarter_start_year,
        (((today.month - 1) - ((today.month - 1) % 3) + 9) % 12) + 1,
        1,
    )
    last_quarter_end = this_quarter_start + timedelta(days=-1)
    this_year_start = date(today.year, 1, 1)
    this_year_end = date(today.year, 12, 31)
    previous_year_start = date(today.year - 1, 1, 1)
    previous_year_end = date(today.year - 1, 12, 31)
    durations_to_dates = {
        'Yesterday': (yesterday, yesterday),
        '7 day': (last_week, today),
        '30 day': (last_month, today),
        '90 day': (last_quarter, today),
        'Month to date': (this_month_start, today),
        'Previous month': (last_full_month_start, last_full_month_end),
        'Quarter to date': (this_quarter_start, today),
        'Previous quarter': (last_quarter_start, last_quarter_end),
        'Year to date': (this_year_start, today),
        'Previous year': (previous_year_start, previous_year_end),
        '1 year': (__date_add_years(today, -1), today),
        '2 year': (__date_add_years(today, -2), today),
        '3 year': (__date_add_years(today, -3), today),
        '5 year': (__date_add_years(today, -5), today),
        '10 year': (__date_add_years(today, -10), today),
        str(today.year): (this_year_start, this_year_end),
        str(__date_add_years(today, -1).year): (
            previous_year_start,
            previous_year_end,
        ),
    }
    for num_years in range(2, 7):
        durations_to_dates[str(__date_add_years(today, -num_years).year)] = (
            date(today.year - num_years, 1, 1),
            date(today.year - num_years, 12, 31),
        )
    return durations_to_dates
possible_dates = get_dates_from_duration()


In [3]:
#store local copy of data in memory to reduce number of calls being made to sever, reduce network traffic

metrics = {}
dimentions = {}

with dw:  # Ensure 'dw' is properly defined and initialized
    realms = dw.describe_realms()
    aggression_units = dw.get_aggregation_units()
    durations = dw.get_durations()

    for realm in realms['label']:  # Ensure 'realms' has a 'label' key and it contains iterable data
        data = []
        for metric_index, metric_row in dw.describe_metrics(realm).iterrows():
            data.append([metric_index, metric_row['label'], metric_row['description']])
        metrics[realm] = data
        data = []
        for dim_index, dim_row in dw.describe_dimensions(realm).iterrows():
            data.append([dim_index,dim_row['label'],dim_row['description']])
        dimentions[realm] = data

In [4]:
import csv
import base64
import sys
import itertools

#User Query | Context (Chart Config, Chart Link, Chart Description) |  Expected Answer  <- try to format data accordingly   + Should I generate sample answers as well as questions?

output_file = 'data_document.csv'
file_header = ["ID","Sample Query", "Duration", "realm", "Metric", "Dimension", "Data Set Type", "Aggregation", "Description","URL"]  # TODO - Finalize format later
count=0

# Create a new CSV file to store data to, and add a header row, defined above
with open(output_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(file_header)
    def add_Query(dur,realm,metric,dimention,dataSetType,aggUnit,set_of_filters,filter_key):
        #QUESTION - Is this correct or does the deccription field contain additional values
        Description = "Description: " +  + "\n" + dimention[2] + "\n"
        # Create URL
        duration = possible_dates[dur]
        start_date =  duration[0].strftime("%Y-%m-%d")
        end_date =  duration[1].strftime("%Y-%m-%d")
        
        filter_value=""
        for filt in set_of_filters:
            filter_value += f"""
                            {{
                                "id": "{metric[1]}={filter_key[filt]}",
                                "value_id": "{filter_key[filt]}",
                                "value_name": "{filt}",
                                "dimension_id": "{metric[1]}",
                                "realms": [
                                    "{realm}"
                                ],
                                "checked": true
                            }},
                            """

        config = """
        {
          "featured": false,
          "trend_line": false,
          "x_axis": {},
          "y_axis": {},
          "legend": {},
          "defaultDatasetConfig": { "display_type": """+graph+""" },
          "swap_xy": false,
          "share_y_axis": false,
          "hide_tooltip": false,
          "show_remainder": false,
          "timeseries": """+str(dataSetType=="timeseries").lower()+""",
          "title": "untitled query",
          "legend_type": "bottom_center",
          "font_size": 3,
          "show_filters": true,
          "show_warnings": true,
          "data_series": {
            "data": [
              {
                "group_by": """+dimention[0]+""",
                "color": "auto",
                "log_scale": false,
                "std_err": false,
                "value_labels": false,
                "display_type": """+graph+""",
                "combine_type": "side",
                "sort_type": "value_desc",
                "ignore_global": false,
                "long_legend": true,
                "x_axis": false,
                "has_std_err": true,
                "trend_line": false,
                "line_type": "Solid",
                "line_width": 2,
                "shadow": false,
                "filters": {
                  "data": [],
                  "total": 0
                },
                "z_index": 0,
                "visibility": null,
                "enabled": true,
                "metric": """+metric[0]+""",
                "realm": """+realm+""",
                "category": """+realm+""",
                "id": """+ '{:0{width}d}'.format(count, width=10)+"""
              }
            ],
            "total": 1
          },
          "aggregation_unit": """+aggUnit+""",
          "global_filters": {
            "data": ["""+filter_value+"""],
            "total": """+count(set_of_filters)+"""
          },
          "timeframe_label": """+dur+""",
          "start_date":"""+start_date+""",
          "end_date": """+end_date+""",
          "start": 0,
          "limit": 10
        }
          """
        # Encode to Base64
        config_encoded = base64.b64encode(config.encode('utf-8')).decode('utf-8') 
        URL = "https://xdmod-dev.ccr.xdmod.org/#main_tab_panel:metric_explorer?config="+config_encoded+"="

        #Generate Question
        #TODO - maybe add several variation questions per type and see if that improves it
        filterList = "" #Filter list will be a list of filters that are applied
        question = "Can you show me "+metric[1]+" per "+(dimention[1] if dimention[1].lower()!='none' else 'all')+" from "+realm+(" totaled " if dataSetType!='timeseries' else " over time ")+"for "+dur +" as a "+graph+" chart "+filterList+"?" 
        writer.writerow(['{:0{width}d}'.format(count, width=10),question,dur, realm, metric[1], dimention[1], dataSetType, aggUnit, Description,URL])


# Loop through all possible permutations of data querying (being duration, realm, metric, dimensions, filters, dataset_type, and aggregation)
    with dw:
        for dur in durations:
            for realm in realms['label']:
                for metric in metrics[realm]: #[code_name, human name, description]
                    for dimention in dimentions[realm]: #[code_name, human name, description]
                        print(dimention)
                        for dataSetType in ["timeseries", "aggregate"]:  # These values are hard coded -> may cause error if new types are later introduced
                            for aggUnit in aggression_units:
                                    for graph in ['line','bar','area','areaspline','scatter','spline','pie']: # NOTE bar is only possible for aggriagte data
                                        if (graph =='pie' and dataSetType=='timeseries'):
                                            exit()
                                        #Count used to generatie unique ID's for each query
                                        count+=1
                                        #Generating filters array
                                        filters=[]
                                        filter_key={}
                                        if dimention[0]!='none':
                                            filter_values = dw.get_filter_values(realm, dimention[0])['label'] # 'resource' also work
                                            for id, label in filter_values.items():
                                                filter_key[label]= id
                                                filters.append(value)
                                        
                                            all_filter_combinations = []
                                            fileter_id_comb=[]
                                            # Iterate over all possible lengths of combinations
                                            for i in range(1, 4):
                                                all_filter_combinations.extend(itertools.combinations(filters, i))
                                            for set_of_filters in all_filter_combinations:
                                                add_Query(dur,realm,metric,dimention,dataSetType,aggUnit,set_of_filters,filter_key)
                                        else:
                                            add_Query(dur,realm,metric,dimention,dataSetType,aggUnit,[],{})
                                        
                                        
                                        
        #                                 # Get the data for a specific set of permutations
        #                                 data = dw.get_data(
        #                                     duration=dur,
        #                                     realm=realm,
        #                                     metric=metrics_label,
        #                                     dimension=dimension_label,
        #                                     filters={},
        #                                     dataset_type=dataSetType,
        #                                     aggregation_unit=aggUnit,
        #                                 )




['none', 'None', 'Summarizes all of the accounts data.']
['resource', 'Resource', 'A resource is a remote computer that can run jobs.']
Can you show me Number of Unique Accounts: Created (Number of Accounts) per Resource from Accounts over time for Yesterday as a line chart ?
['ANL DTF', 'ANL VIS', 'ASTA', 'Bridges 2 EM', 'Bridges 2 GPU', 'Bridges 2 RM', 'Bridges 2 Storage', 'Bridges2 GPU AI', 'CALTECH DTF', 'CCT LSU SUPERMIC', 'COLORADO BOULDER RMACC SUMMIT', 'COVID 19 Consortium', 'credits.allocations.access ci.org', 'ECSS', 'Expanse', 'Expanse GPU', 'Expanse Projects Storage', 'GATECH KEENELAND', 'GATECH KIDS', 'IACS Ookami', 'Indiana U Jetstream2', 'Indiana U Jetstream2 GPU', 'Indiana U Jetstream2 Large Memory', 'Indiana U Jetstream2 Storage', 'IU AVIDD IA32', 'IU BIGRED*', 'IU COLLECTIONS', 'IU CONDOR', 'IU HPSS', 'IU MASON', 'IU QUARRY', 'IU TACC JETSTREAM', 'IU TACC JETSTREAM STORAGE', 'IU TIGER', 'JHU Rockfish A100', 'JHU Rockfish LM', 'JHU Rockfish RM', 'KyRIC', 'LONI QUEENBEE

SystemExit: 

/Users/tim_main/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3406: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
